In [3]:
#Importing Required Libraries
!pip install fastapi uvicorn[standard] joblib scikit-learn web3
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import joblib
import numpy as np
from web3 import Web3
import datetime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 580.7/580.7 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.5/100.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.8/459.8 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.6/452.6 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.1/303.1 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.3

In [4]:
# Initializing FastAPI App

app = FastAPI()

In [5]:
# Enabling CORS for Frontend Communication
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Replace '*' with frontend domain in production
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [10]:
#Loading Trained Model and Scaler

model = joblib.load("/content/best_mental_health_model.joblib")   # Machine learning model
scaler = joblib.load("/content/mental_health_scaler.joblib")      # StandardScaler used during training


In [11]:
# Defining Input Schema for Survey Data

class SurveyInput(BaseModel):
    Age: int
    Gender: str
    self_employed: str
    family_history: str
    work_interfere: str
    no_employees: str
    remote_work: str
    tech_company: str
    benefits: str
    care_options: str
    wellness_program: str
    seek_help: str
    anonymity: str
    leave: str
    mental_health_consequence: str
    phys_health_consequence: str
    coworkers: str
    supervisor: str
    mental_health_interview: str
    phys_health_interview: str
    mental_vs_physical: str
    obs_consequence: str

In [12]:
# Preprocessing User Input for Prediction
def preprocess_input(data: dict):
    ordered_features = [
        data["Age"],
        hash(data["Gender"]) % 10,
        hash(data["self_employed"]) % 10,
        hash(data["family_history"]) % 10,
        hash(data["work_interfere"]) % 10,
        hash(data["no_employees"]) % 10,
        hash(data["remote_work"]) % 10,
        hash(data["tech_company"]) % 10,
        hash(data["benefits"]) % 10,
        hash(data["care_options"]) % 10,
        hash(data["wellness_program"]) % 10,
        hash(data["seek_help"]) % 10,
        hash(data["anonymity"]) % 10,
        hash(data["leave"]) % 10,
        hash(data["mental_health_consequence"]) % 10,
        hash(data["phys_health_consequence"]) % 10,
        hash(data["coworkers"]) % 10,
        hash(data["supervisor"]) % 10,
        hash(data["mental_health_interview"]) % 10,
        hash(data["phys_health_interview"]) % 10,
        hash(data["mental_vs_physical"]) % 10,
        hash(data["obs_consequence"]) % 10
    ]
    return scaler.transform([ordered_features])  # Apply scaling to input

In [13]:
#Placeholder Function for Blockchain Logging

def log_to_blockchain(data, prediction):
    try:
        # Connect to local Ethereum blockchain (Ganache)
        web3 = Web3(Web3.HTTPProvider("http://127.0.0.1:7545"))

        if web3.isConnected():
            print("🟢 Connected to Blockchain")
            print({
                "prediction": prediction,
                "timestamp": str(datetime.datetime.utcnow()),
                "data": data
            })
            # TODO: Replace with actual smart contract call
        else:
            print("🔴 Blockchain not connected")

    except Exception as e:
        print(f"Blockchain logging error: {e}")

In [14]:
# Predict Endpoint (POST Request)
@app.post("/predict")
def predict(input_data: SurveyInput):
    try:
        # Convert user input to dictionary format
        data = input_data.dict()

        # Preprocess the input
        processed_input = preprocess_input(data)

        # Make prediction using ML model
        prediction = model.predict(processed_input)[0]

        # Log to blockchain (placeholder)
        log_to_blockchain(data, str(prediction))

        # Return result to frontend
        return {
            "status": "success",
            "prediction": str(prediction)
        }

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

In [15]:
# Health Check Endpoint (GET Request)

@app.get("/health")
def health_check():
    return {"status": "Backend is running ✅"}